# Latent space vizualization


In [1]:
import json
import integrated_cell
from integrated_cell import model_utils, utils
import os
import numpy as np
import torch

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import PIL
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

# 
from integrated_cell.utils.plots import tensor2im, imshow

# tricking Pytorch environment which GPU to use
gpu_ids = [7]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(ID) for ID in gpu_ids])
if len(gpu_ids) == 1:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

torch.cuda.empty_cache()
    
#load the reference model
parent_dir = '/allen/aics/modeling/gregj/results/integrated_cell/'

# reference model is trained with a timestamp
ref_model_dir = '/allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-11-27-22:27:04'
# iteration 94544, pick your favorite one, perhaps the one with the lowest validation error.  
ref_suffix = '_94544'    

networks, dp_ref, args_ref = utils.load_network_from_dir(ref_model_dir, parent_dir, suffix=ref_suffix)

ref_enc = networks['enc']
ref_dec = networks['dec']

#load the target model
target_model_dir = "/allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-10-22-15:24:09/"
target_suffix = '_93300'

networks, dp_target, args_target = utils.load_network_from_dir(target_model_dir, parent_dir, suffix=target_suffix)
    
target_enc = networks['enc']
target_dec = networks['dec']

# where to save
results_dir = '{}/results/latent_space_vizualization/'.format(parent_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
print("Results dir: {}".format(results_dir))

save_dir = results_dir

recon_loss = utils.load_losses(args_target)['crit_recon']

no changes


/allen/aics/modeling/ritvik/projects/pytorch_integrated_cell_2021_workshop/pytorch_integrated_cell/integrated_cell/utils/utils.py:75: UserWarning: args file exists and overwrite is not set to True. Using existing args file.
  "args file exists and overwrite is not set to True. Using existing args file."
/allen/aics/modeling/ritvik/projects/pytorch_integrated_cell_2021_workshop/pytorch_integrated_cell/integrated_cell/utils/utils.py:75: UserWarning: args file exists and overwrite is not set to True. Using existing args file.
  "args file exists and overwrite is not set to True. Using existing args file."


loading from /allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-11-27-22:27:04/ref_model/enc_94544.pth
loading from /allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-11-27-22:27:04/ref_model/dec_94544.pth
no changes


/allen/aics/modeling/ritvik/projects/pytorch_integrated_cell_2021_workshop/pytorch_integrated_cell/integrated_cell/utils/utils.py:75: UserWarning: args file exists and overwrite is not set to True. Using existing args file.
  "args file exists and overwrite is not set to True. Using existing args file."


loading from /allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-10-22-15:24:09/ref_model/enc_93300.pth
loading from /allen/aics/modeling/gregj/results/integrated_cell/test_cbvae_3D_avg_inten/2019-10-22-15:24:09/ref_model/dec_93300.pth
Results dir: /allen/aics/modeling/gregj/results/integrated_cell//results/latent_space_vizualization/


In [2]:
import pandas as pd

## Get metadata for all splits

In [3]:
dfs_split = {
    split: pd.concat(
        [
            dp_ref.csv_data.loc[
                dp_ref.data[split]["inds"],
                ["CellId", "PlateId", "CellIndex", "StructureDisplayName"]
            ].reset_index(
            ).rename(
                columns={"index":"UnsplitCsvIndex"}
            ),
            pd.DataFrame({"split":[split]*len(dp_ref.data[split]["inds"])})
        ],
        axis=1
    ) for split in dp_ref.data.keys()
}

In [4]:
dfs_split['test']




,UnsplitCsvIndex,CellId,PlateId,CellIndex,StructureDisplayName,split
0,14,81,3500001454,7,Lysosome,test
1,32,149,3500001454,11,Lysosome,test
2,33,155,3500001454,5,Lysosome,test
3,42,205,3500001454,5,Lysosome,test
4,50,235,3500001454,11,Lysosome,test
...,...,...,...,...,...,...
4740,49330,151618,3500002027,7,Control - DNA,test
4741,49331,151618,3500002027,7,Control - Memb,test
4742,49332,151618,3500002027,7,Control - Blank,test
4743,49333,151618,3500002027,7,Control - Noise,test


## Get mito annotations for each split

In [ ]:
# from integrated_cell.utils.utils import save_load_mitosis_annotations
# mito_annots = save_load_mitosis_annotations(dp_ref, save_dir=None)

In [5]:
dfs_mito = {
    split: pd.DataFrame(
        {
            feat:dp_ref.data["test"][feat] for feat in ["inds", "CellId", "mito_state_resolved"]
        }
    ).rename(
        columns={"inds":"UnsplitCsvIndex"}
    ) for split in dp_ref.data.keys()
}

In [6]:
dfs_mito['test'].mito_state_resolved.unique()


array(['M0', 'M6/M7', 'M4/M5', 'M3', 'M1/M2', 'u'], dtype=object)

## Get features for all splits together (unsplit df)

In [7]:
import pandas as pd

with open('/raid/shared/ipp/scp_19_04_10/feats_out_with_units.json') as f:
    feats = json.load(f)

df_feats_all = pd.DataFrame.from_dict(feats["rows_with_units"])
my_feats = ["CellId", "dna_volume", "cell_volume"]
df_feats = df_feats_all[my_feats].copy()

In [9]:
df_feats["cell_height"] = df_feats_all["cell_position_highest_z"] - df_feats_all["cell_position_lowest_z"]
df_feats["dna_height"] = df_feats_all["dna_position_highest_z"] - df_feats_all["dna_position_lowest_z"]

In [10]:
df_feats_all


,imsize_orig,com,angle,flipdim,imsize_registered,dna_cell_struct_cyto_distal_ratio,dna_cell_struct_nuc_distal_ratio,dna_cell_struct_cell_distal_ratio,dna_cell_struct_cyto_z_ratio,dna_cell_struct_nuc_z_ratio,...,cell_texture_haralick_inv_diff_moment,cell_texture_haralick_sum_avg,cell_texture_haralick_sum_var,cell_texture_haralick_sum_entropy,cell_texture_haralick_entropy,cell_texture_haralick_diff_var,cell_texture_haralick_diff_entropy,cell_texture_haralick_info_corr1,cell_texture_haralick_info_corr2,CellId
0,"[6, 233, 345, 70]","[0.0, 95.73798643341664, 139.71142449125313, 3...","[[0, -341.75582526118234]]","[False, False, True, False]","[6, 46, 162, 106]",-0.678253,-0.858508,0.710400,0.647402,0.335951,...,0.391685,11.219378,70.785157,4.669716,7.000211,0.002443,2.853355,-0.136620,0.787920,3
1,"[6, 233, 345, 70]","[0.0, 174.5516260117591, 90.00035432329787, 34...","[[0, -329.28894094968496]]","[False, True, False, False]","[6, 44, 140, 82]",-0.379048,-0.861645,0.804327,0.764466,0.217478,...,0.372715,13.767075,145.516059,5.041606,7.508996,0.000778,3.049410,-0.170585,0.857710,4
2,"[6, 233, 345, 70]","[0.0, 59.68728602659114, 101.54515956057251, 3...","[[0, -99.35645917871796]]","[False, False, False, False]","[6, 52, 104, 64]",-0.362142,-0.822053,0.769682,0.738891,0.312548,...,0.340465,14.691660,156.808053,5.129517,7.744635,0.001533,3.165321,-0.155474,0.843536,8
3,"[6, 233, 345, 70]","[0.0, 167.76100180234306, 48.8582156803845, 35...","[[0, -186.2046032385802]]","[False, True, True, False]","[6, 50, 110, 54]",-0.542339,-0.785274,0.862652,0.847001,-0.198947,...,0.371594,15.811755,174.186657,5.264366,7.781250,0.002252,3.057061,-0.198706,0.898151,9
4,"[6, 233, 345, 70]","[0.0, 82.90367449254096, 111.75973648813891, 3...","[[0, -205.68844110600955]]","[False, True, True, False]","[6, 36, 100, 90]",-0.084949,-0.809483,0.850606,0.279812,-0.677044,...,0.327381,15.217116,266.227963,5.161616,7.830178,0.000883,3.336896,-0.147983,0.834868,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39185,"[6, 233, 345, 65]","[0.0, 180.47328432687664, 86.54450823630413, 3...","[[0, -212.0118390169808]]","[False, False, False, False]","[6, 48, 124, 82]",-0.154522,0.849174,-0.969223,0.962999,-0.008615,...,0.436732,11.057878,145.935109,4.639912,6.847553,0.002150,2.990089,-0.152317,0.810255,151639
39186,"[6, 233, 345, 65]","[0.0, 166.78789497472982, 127.12941885469313, ...","[[0, -51.06735666944357]]","[False, False, False, False]","[6, 50, 104, 86]",0.194069,0.885184,-0.970856,0.673005,-0.040778,...,0.420820,12.155386,131.925161,4.784324,7.005031,0.001388,2.866283,-0.181705,0.854017,151641
39187,"[6, 233, 345, 65]","[0.0, 117.11161246656138, 168.3681739560168, 3...","[[0, -296.44261708043825]]","[False, False, False, False]","[6, 50, 140, 82]",-0.195323,0.843532,-0.976143,0.426922,-0.034855,...,0.454499,10.659366,96.663681,4.611459,6.695093,0.002452,2.767165,-0.183544,0.847190,151642
39188,"[6, 233, 345, 65]","[0.0, 67.76593546336035, 184.76893975520585, 2...","[[0, -19.700272475770966]]","[False, False, False, False]","[6, 68, 98, 68]",0.252698,0.848712,-0.924396,-0.421818,0.184838,...,0.538873,8.850230,93.723706,4.258818,6.015484,0.002161,2.541985,-0.201376,0.848389,151643


## Find the embeddings for each split

In [11]:
from integrated_cell.metrics.embeddings_reference import get_latent_embeddings
# running each of the test cells 
embeds_test = get_latent_embeddings(
    ref_enc,
    ref_dec,
    dp_ref,
    recon_loss,
    modes=['test'],
    batch_size=32,
)

100%|██████████| 149/149 [23:03<00:00,  9.29s/it]


In [ ]:
dfs_embeds = {
    split: pd.DataFrame(
        embeds_test[split]["ref"]["mu"].numpy(),
        columns=[f"mu_{i}" for i in range(embeds_test["test"]["ref"]["mu"].numpy().shape[1])]
    ) for split in ['test']
}

In [ ]:
for split, df_embed in dfs_embeds.items():
    assert len(dfs_embeds[split]) == len(dfs_split[split])
    dfs_embeds[split]["UnsplitCsvIndex"] = dfs_split[split]["UnsplitCsvIndex"]

In [ ]:
# this is just for test set until i can run all splits

# embeds_test = get_latent_embeddings(ref_enc, ref_dec, dp_ref, recon_loss, modes=['test'], batch_size=32,)

df_embeds_test = pd.DataFrame(
    embeds_test["test"]["ref"]["mu"].numpy(),
    columns=[f"mu_{i}" for i in range(embeds_test["test"]["ref"]["mu"].numpy().shape[1])]
)

assert len(df_embeds_test) == len(dfs_split["test"])
df_embeds_test["UnsplitCsvIndex"] = dfs_split["test"]["UnsplitCsvIndex"]

## Merge embeddings in to metadata

In [ ]:
dfs_split = {
    split: df_split.merge(df_feats) for split, df_split in dfs_split.items()
}

In [ ]:
dfs_split

In [ ]:
# need to fix up to do all splits at once

In [ ]:
df_embeddings_plus_meta_test = dfs_split["test"].merge(
    dfs_mito["test"]
).merge(
    df_feats
).merge(
    df_embeds_test
)

df_embeddings_plus_meta_test_all_feats = dfs_split["test"].merge(
    dfs_mito["test"]
).merge(
    df_feats_all
).merge(
    df_embeds_test
)

# df_embeddings_plus_meta_test.to_csv("embeddings_plus_meta_test.csv")

In [ ]:
df_embeddings_plus_meta_test

## find top latent dims and drop others for grid plots

In [ ]:
feature_corrs = df_embeddings_plus_meta_test[
    [c for c in df_embeddings_plus_meta_test.columns if "mu" in c or c in df_feats.drop(columns=["CellId"]).columns]
].corr(method="spearman")

In [ ]:
feature_corrs.columns

In [ ]:
top_mu_dims = list(
    df_embeddings_plus_meta_test[
        [c for c in df_embeddings_plus_meta_test.columns if "mu" in c]
    ].abs().mean().sort_values().index
)
top_mu_dims.reverse()
top_mu_dims[0:10]

In [ ]:
cols_to_drop = [
    c for c in df_embeddings_plus_meta_test.columns if "mu" in c and c not in top_mu_dims
]
df_embeddings_plus_meta_test_top = df_embeddings_plus_meta_test.drop(columns=cols_to_drop)

## plots

In [ ]:
# drop missing structure labels
df_embeddings_plus_meta_test = df_embeddings_plus_meta_test.dropna()

# drop controls
df_embeddings_plus_meta_test = df_embeddings_plus_meta_test[
    ~df_embeddings_plus_meta_test.StructureDisplayName.str.startswith("Control")
]

## plot mito annotations

In [ ]:
df_mito_search = df_embeddings_plus_meta_test_all_feats.dropna().copy()
df_mito_search = df_mito_search[df_mito_search["mito_state_resolved"] != 'u']

df_mito_search["mito_state_int"] = df_mito_search["mito_state_resolved"].map(
    {'M0':0, 'M1/M2':1, 'M3':2, 'M4/M5':3, 'M6/M7':4}
)
# df_mito_search = df_mito_search[df_mito_search["mito_state_int"] != 0]

mu_cols = [c for c in df_embeds_test.columns if c.startswith("mu_") and c != "CellId"]
mito_corrs = df_mito_search[mu_cols+["mito_state_int"]].corr(method="spearman")
mito_corrs = mito_corrs.drop(mu_cols, axis="columns").drop(["mito_state_int"], axis="rows")

In [ ]:
df = pd.DataFrame()
df['X1'] = np.random.permutation(mito_corrs['mito_state_int'])
df.cumsum().plot()
mito_corrs['mito_state_int'].cumsum().plot()


In [ ]:
df_embeddings_plus_meta_test["mu_0"][0]                             

In [ ]:
plt.figure(figsize=(4,4))

g = sns.lineplot(
    x=mito_corrs.abs().rank(ascending=False)["mito_state_int"],
    y=mito_corrs.abs()["mito_state_int"]
);

g.set(
#     xlim=(-5, 3),
#     ylim=(-4, 5),
    xlabel="Dimension rank",
    ylabel="Spearman corr. with mitotic state",
);

sns.despine()

plt.savefig('mito_dims_ranked.png', dpi=300, bbox_inches = "tight")

In [ ]:
top_mito_dims = list(mito_corrs.abs().squeeze().sort_values().index)
top_mito_dims.reverse()
top_mito_dims[0:10]

In [ ]:
mito_dims_compared_to_overall_mu_ranks = [
    (dim, i, top_mu_dims.index(dim))
    for i, dim in enumerate(top_mito_dims[:8])]

mito_dims_compared_to_overall_mu_ranks

In [ ]:
pd.DataFrame(mito_dims_compared_to_overall_mu_ranks, columns=["dim", "mito rank", "overall rank"])

## plots

In [ ]:
df_embeddings_plus_meta_test_pretty_names = df_embeddings_plus_meta_test.rename(
    columns={
        "mito_state_resolved": "Mitotic state",
        'dna_volume': 'DNA volume',
        'cell_volume': 'Cell volume',
        'dna_height': 'DNA height',
        'cell_height': 'Cell height'
    }
)

pixel_length_in_micrometers = 0.29
df_embeddings_plus_meta_test_pretty_names['DNA volume (μm^3)'] = df_embeddings_plus_meta_test_pretty_names['DNA volume']*(pixel_length_in_micrometers**3)
df_embeddings_plus_meta_test_pretty_names['DNA height (μm)'] = df_embeddings_plus_meta_test_pretty_names['DNA height']*pixel_length_in_micrometers
df_embeddings_plus_meta_test_pretty_names['Cell volume (μm^3)'] = df_embeddings_plus_meta_test_pretty_names['Cell volume']*(pixel_length_in_micrometers**3)
df_embeddings_plus_meta_test_pretty_names['Cell height (μm)'] = df_embeddings_plus_meta_test_pretty_names['Cell height']*pixel_length_in_micrometers

### mito plots

#### by top 2 dims that separate mito states

In [ ]:
plt.figure(figsize=(3,3))

plt_mito = sns.scatterplot(
    data=df_embeddings_plus_meta_test_pretty_names[
        df_embeddings_plus_meta_test_pretty_names["Mitotic state"] != 'u'
    ].sort_values(
        by="Mitotic state"
    ),
    x="mu_452",
    y="mu_419",
    hue="Mitotic state",
    hue_order=['M0', 'M1/M2', 'M3', 'M4/M5', 'M6/M7'],
    palette=[("lightgrey")] + sns.color_palette("tab10")[:4],
    linewidth=0,
    alpha = 0.5,
    s=20,
)

plt_mito.set(
    xlabel="$\mu_{452}$",
    ylabel="$\mu_{419}$",
);

plt.legend(bbox_to_anchor=(1.0, 0.7), frameon=False)
sns.despine()

plt.savefig('latent_plot_mito_top_mito_dims.png', dpi=300, bbox_inches = "tight")

#### by top two overall dims

In [ ]:
plt.figure(figsize=(3,3))

plt_mito = sns.scatterplot(
    data=df_embeddings_plus_meta_test_pretty_names[
        df_embeddings_plus_meta_test_pretty_names["Mitotic state"] != 'u'
    ].sort_values(
        by="Mitotic state"
    ),
    x="mu_71",
    y="mu_419",
    hue="Mitotic state",
    hue_order=['M0', 'M1/M2', 'M3', 'M4/M5', 'M6/M7'],
    palette=[("lightgrey")] + sns.color_palette("tab10")[:4],
    linewidth=0,
    alpha = 0.5,
    s=20,
)

plt_mito.set(
    xlabel="$\mu_{71}$",
    ylabel="$\mu_{419}$",
);

plt.legend(bbox_to_anchor=(1.0, 0.7), frameon=False)
sns.despine()

plt.savefig('latent_plot_mito_top_mu_overall_dims.png', dpi=300, bbox_inches = "tight")

### grid plots

In [ ]:
pretty_hue_names = {
    'DNA volume (μm^3)': 'DNA volume (μm$^3$)',
    'DNA height (μm)': 'DNA height (μm)',
    'Cell volume (μm^3)': 'Cell volume (μm$^3$)',
    'Cell height (μm)': 'Cell height (μm)',
}

In [ ]:
pretty_mu_names = {
    'mu_71': "$\mu_{71}$",
    'mu_419': "$\mu_{419}$",
    'mu_188': "$\mu_{188}$",
    'mu_465': "$\mu_{465}$",
}
pretty_mu_names

In [ ]:
for hue, hue_pretty_name in pretty_hue_names.items():

    hue_norm_tail = 0.01

    df_plot = df_embeddings_plus_meta_test_pretty_names[
        df_embeddings_plus_meta_test_pretty_names["Mitotic state"] != 'u'
    ]

    g = sns.PairGrid(
        df_plot,
        height=1.5,
        vars=top_mu_dims[:4]
    )
    g.map_diag(sns.kdeplot, color=".2", shade=False)
    g.map_offdiag(
        sns.scatterplot,
        hue=df_plot[hue],
        palette="viridis",
        s=20,
        linewidth=0,
        alpha=0.5,
        hue_norm=(np.quantile(df_plot[hue], hue_norm_tail), np.quantile(df_plot[hue], 1-hue_norm_tail))
    );

    plt.legend(title=hue_pretty_name, bbox_to_anchor=(1.01, 1), frameon=False)
#     g._legend.set_title(new_title)
    
    for i in range(4):
        for j in range(4):
            xlabel = g.axes[i][j].get_xlabel()
            ylabel = g.axes[i][j].get_ylabel()
            if xlabel in pretty_mu_names.keys():
                g.axes[i][j].set_xlabel(pretty_mu_names[xlabel])
            if ylabel in pretty_mu_names.keys():
                g.axes[i][j].set_ylabel(pretty_mu_names[ylabel])
    
    
    plt.subplots_adjust(top=0.95)
#     g.fig.suptitle(f"color = {hue_pretty_name}");
    g.fig.savefig(f"latent_top_mu_dims_gridplot_color_{hue.lower().replace(' ', '_')}.png", dpi=300, bbox_inches = "tight")

### cell volume and height solo for main figure

In [ ]:
plt.figure(figsize=(3,3))

hue_norm_tail = 0.01

plt_mito = sns.scatterplot(
    data=df_embeddings_plus_meta_test_pretty_names[
        df_embeddings_plus_meta_test_pretty_names["Mitotic state"] != 'u'
    ].sort_values(
        by="Cell height (μm)"
    ),
    x="mu_71",
    y="mu_419",
    hue="Cell height (μm)",
    palette="viridis",
    linewidth=0,
    alpha = 0.5,
    s=20,
    hue_norm=(
        np.quantile(df_embeddings_plus_meta_test_pretty_names["Cell height (μm)"], hue_norm_tail),
        np.quantile(df_embeddings_plus_meta_test_pretty_names["Cell height (μm)"], 1-hue_norm_tail)
    )
)

plt_mito.set(
    xlabel="$\mu_{71}$",
    ylabel="$\mu_{419}$",
);

plt.legend(bbox_to_anchor=(1.0, 0.7), frameon=False)
sns.despine()

plt.savefig('latent_plot_mito_cell_height_alone.png', dpi=300, bbox_inches = "tight")

In [ ]:
plt.figure(figsize=(3,3))

plt_mito = sns.scatterplot(
    data=df_embeddings_plus_meta_test_pretty_names[
        df_embeddings_plus_meta_test_pretty_names["Mitotic state"] != 'u'
    ].sort_values(
        by="Cell volume (μm^3)"
    ),
    x="mu_71",
    y="mu_419",
    hue="Cell volume (μm^3)",
    palette="viridis",
    linewidth=0,
    alpha = 0.5,
    s=20,
    hue_norm=(
        np.quantile(df_embeddings_plus_meta_test_pretty_names["Cell volume (μm^3)"], hue_norm_tail),
        np.quantile(df_embeddings_plus_meta_test_pretty_names["Cell volume (μm^3)"], 1-hue_norm_tail)
    )
)

plt_mito.set(
    xlabel="$\mu_{71}$",
    ylabel="$\mu_{419}$",
);

plt.legend(bbox_to_anchor=(1.0, 0.7), frameon=False)
sns.despine()

plt.savefig('latent_plot_mito_cell_volume_alone.png', dpi=300, bbox_inches = "tight")

## for the top N latent dims, find the top features correlated with them

In [ ]:
mu_cols = [c for c in df_embeds_test.columns if c.startswith("mu_") and c != "CellId"]
feat_cols = [c for c in df_feats_all.columns if (c.startswith("dna_") or c.startswith("cell_"))]
corrs_all = df_embeddings_plus_meta_test_all_feats[mu_cols+feat_cols].corr(method="spearman")

In [ ]:
corrs_all_cross = corrs_all.drop(mu_cols, axis="rows").drop(feat_cols, axis="columns")

In [ ]:
top_mu_dims = list(
    df_embeddings_plus_meta_test[mu_cols].abs().mean().sort_values().index
)
top_mu_dims.reverse()

In [ ]:
plt.figure(figsize=(4,2))

g = sns.lineplot(
    x=df_embeddings_plus_meta_test[mu_cols].abs().mean().rank(ascending=False),
    y=df_embeddings_plus_meta_test[mu_cols].abs().mean()
);

g.set(
#     xlim=(-5, 3),
#     ylim=(-4, 5),
    xlabel="Reference latent space dimension rank",
    ylabel="Mean absolute value",
);
sns.despine()

plt.savefig('mu_overall_dims_ranked.png', dpi=300, bbox_inches = "tight")

In [ ]:
g = sns.clustermap(
    corrs_all_cross[top_mu_dims[:32]].abs(),
    col_cluster=False,
    figsize=(10,20),
    cmap="Blues",
    vmin=0,
#     center=0,
    vmax=1,
    xticklabels=True,
    yticklabels=True,
);

g.savefig(f"heatmap_mu_vs_feats_vertical.png", dpi=300, bbox_inches = "tight")

In [ ]:
top_mu_dims[:32]

In [ ]:
g = sns.clustermap(
    corrs_all_cross.T.abs(),
    row_cluster=False,
    figsize=(10,30),
    cmap="Blues",
    vmin=0,
#     center=0,
    vmax=.1,
    xticklabels=True,
    yticklabels=True,
);
g.savefig(f"heatmap_mu_vs_feats_horizontal.png", dpi=300, bbox_inches = "tight")

In [ ]:
N_latent_dims = 8
K_feats_per_dim = 3

mu_feat_match_dict = {}
for c in corrs_all_cross[top_mu_dims[:N_latent_dims]].columns:
    top_dims_for_c = list(corrs_all_cross[top_mu_dims][c].abs().sort_values()[-K_feats_per_dim:].index)
    top_dims_for_c.reverse()
    mu_feat_match_dict[c] = top_dims_for_c
    
df_mu_feat_match = pd.DataFrame(mu_feat_match_dict)

In [ ]:
df_mu_feat_match

## draw samples from latent space along top N interesting dimensions

In [ ]:
for dim in df_mu_feat_match.columns:
    int_dim = int(dim.split("_")[-1])
    top_feat_for_dim = df_mu_feat_match.loc[0, dim]
    
    latent_walk_path = np.float32(df_embeds_test[dim].std()*np.linspace(-10,10,9))
    latent_walk_path = torch.from_numpy(latent_walk_path)

    latent_input = torch.zeros([len(latent_walk_path), 512], dtype=torch.float32)
    latent_input[:,int_dim] = latent_walk_path
    latent_input = latent_input.cuda()
    
    walk_output = ref_dec(latent_input)
    walk_output = walk_output.cpu().data
    walk_im = tensor2im(walk_output)
    
    pil_im = Image.fromarray(np.uint8(walk_im*255))
    pil_im.save(f"latent_walk_{dim}_top_feat_{top_feat_for_dim}.png","PNG")
    
    print(f"{dim}, top feature = {top_feat_for_dim}")
    display(pil_im)
    print("")

In [ ]:
walk_output.shape


In [ ]:
df_embeds_test[[c for c in df_em]]

In [ ]:
df_embeds[["mu_71", "mu_419"]].std()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
pca = PCA()
pca.fit(scale(df_feats_all[feat_cols].dropna()))

In [ ]:
plt.figure(figsize=(4,2))

g = sns.lineplot(
    x=pd.DataFrame(
        pca.explained_variance_ratio_, columns=["Feature PCA explained variance"]
    ).index,
    y=pd.DataFrame(
        pca.explained_variance_ratio_, columns=["Feature PCA explained variance"]
    )["Feature PCA explained variance"]
);

g.set(
#     xlim=(0, 12),
#     ylim=(-4, 5),
    xlabel="Feature PC rank",
    ylabel="Explained variance",
);
sns.despine()

plt.savefig('pca_feats_explained_variance.png', dpi=300, bbox_inches = "tight")

In [ ]:
pd.DataFrame(
    np.cumsum(pca.explained_variance_ratio_), columns=[
    "Feature PCA explained variance"
    ]
)["Feature PCA explained variance"]

In [ ]:
mu_corrs = df_embeddings_plus_meta_test_all_feats.corr()

plt.figure(figsize=(8,8))
g = sns.heatmap(
    mu_corrs.abs(),
    cmap="Blues",
    vmin=-.1,
    vmax=.1,
    xticklabels=True,
    yticklabels=True,
    square=True,
    cbar_kws={"shrink": .82}
)

plt.savefig('latent_dim_corrs.png', dpi=300, bbox_inches = "tight")